# Modelo CountVectorizer
Crearemos un modelo Vectorizador de Texto (CountVectorizer) que convertira el texto de la columna 'genres' en valores numericos, donde cada palabra unica presente en el texto tendra un numero y contara su frecuencia. Cada juego se representa como un vector donde cada posicion corresponde a la frecuencia de una palabra. Esta representación vectorial es necesaria para calcular la **similitud del coseno** que es clave para determinar qué tan parecidos son los juegos entre sí. Esto se utiliza para generar recomendaciones, ya que los juegos con vectores similares son considerados como recomendaciones potenciales.
lo consigue midiendo el coseno del ángulo entre dos vectores. Cuanto más cercano a 1, más similares son los vectores.

## Requisitos

🔎Asegúrate de instalar la biblioteca de scikit-learn

Para instalar esta biblioteca debes abrir una terminal o ventana de línea de comandos y ejecutar el siguiente comando:

<span style="background-color: #f2f2f2; color: black;">pip install sklearn</span>

In [14]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer #---> vectorizador de texto
from sklearn.metrics.pairwise import cosine_similarity #---> Metrica de similitud del coseno

In [2]:
# Cargar el conjunto de datos
steam_games = pd.read_parquet('C:\\Users\\Juampi\\Desktop\\Henry\\PI_MLOps\\Datasets\\archivos_procesados\\steam_games_cleaned.parquet')

In [3]:
steam_games

,genres,name,price,item_id,developer,release_year
0,"[Strategy, Action, Indie, Casual, Simulation]",Lost Summoner Kitty,4.99,761140.0,Kotoshiro,2018
1,"[Free to Play, Strategy, Indie, RPG]",Ironbound,NaN,643980.0,Secret Level SRL,2018
2,"[Free to Play, Simulation, Sports, Casual, Indie]",Real Pool 3D - Poolians,NaN,670290.0,Poolians.com,2017
3,"[Action, Adventure, Casual]",弹炸人2222,0.99,767400.0,彼岸领域,2017
4,"[Action, Indie, Casual, Sports]",Log Challenge,2.99,773570.0,None,<NA>
...,...,...,...,...,...,...
32130,"[Strategy, Indie, Casual, Simulation]",Colony On Mars,1.99,773640.0,"Nikita ""Ghost_RUS""",2018
32131,"[Strategy, Indie, Casual]",LOGistICAL: South Africa,4.99,733530.0,Sacada,2018
32132,"[Indie, Simulation, Racing]",Russian Roads,1.99,610660.0,Laush Dmitriy Sergeevich,2018
32133,"[Indie, Casual]",EXIT 2 - Directions,4.99,658870.0,"xropi,stev3ns",2017


In [4]:
# Seleccionar las columnas necesarias
modelo_df = steam_games.loc[:, ["genres", "item_id", "name"]]

# Convertir 'item_id' a tipo entero
modelo_df["item_id"] = modelo_df["item_id"].astype(int)


In [5]:
# Asignamos la columna 'Genres' a una variable para luego tratarla y limpiar caracteres
genres_variable = modelo_df['genres'].copy()

# Convertimos a cadena antes de reemplazar los corchetes que queremos sacar
genres_variable = genres_variable.apply(lambda x: str(x).replace('[', '').replace(']', ''))

In [6]:
# Eliminamos las comillas y agregamos comas entre los géneros para separalos
genres_variable = genres_variable.apply(lambda x: ', '.join(filter(None, map(str.strip, x.split("'")))) if isinstance(x, str) else x)

# Reemplazamos la columna 'Genres' en el DataFrame original con la nueva columna tratada
modelo_df['genres'] = genres_variable

# Imprimimos para ver como quedo la columna después de reemplazar
print(modelo_df['genres'])

0              Strategy, Action, Indie, Casual, Simulation
1                       Free to Play, Strategy, Indie, RPG
2          Free to Play, Simulation, Sports, Casual, Indie
3                                Action, Adventure, Casual
4                            Action, Indie, Casual, Sports
                               ...                        
32130                  Strategy, Indie, Casual, Simulation
32131                              Strategy, Indie, Casual
32132                            Indie, Simulation, Racing
32133                                        Indie, Casual
32134    Early Access, Adventure, Indie, Action, Simula...
Name: genres, Length: 32133, dtype: object


In [7]:
# Vamos a cambiar unos errores que hay en la columna Genres
# como vemos hay unos errores de caracteres, que vamos a solucionar de la siguiente manera
modelo_df['genres'] = modelo_df['genres'].replace('Design &amp; Illustration', 'Design & Illustration')
modelo_df['genres'] = modelo_df['genres'].replace('Animation &amp; Modeling', 'Animation & Modeling')

In [8]:
print(modelo_df['genres'].unique())

['Strategy, Action, Indie, Casual, Simulation'
 'Free to Play, Strategy, Indie, RPG'
 'Free to Play, Simulation, Sports, Casual, Indie' ...
 'Early Access, Action, Indie, Simulation, RPG'
 'Free to Play, Strategy, Action, Massively Multiplayer, RPG'
 'Early Access, Adventure, Indie, Action, Simulation']


In [10]:
# Crear un vectorizador de texto
cv = CountVectorizer()
cv.fit_transform(modelo_df['genres']).toarray().shape

(32133, 29)

In [11]:
# Se generan los vectores a comparar 
vectores = cv.fit_transform(modelo_df['genres']).toarray()

In [12]:
# Mostramos como quedo Nuestra variable vectores
vectores

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [16]:


# Verificar si hay NaN
nan_indices = np.isnan(vectores)
print("Índices con NaN:", nan_indices)
print("Valores con NaN:", vectores[nan_indices])

# Verificar si hay infinitos
inf_indices = np.isinf(vectores)
print("Índices con infinitos:", inf_indices)
print("Valores con infinitos:", vectores[inf_indices])


Índices con NaN: [[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Valores con NaN: []
Índices con infinitos: [[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Valores con infinitos: []


In [13]:
# Calcular la similitud del coseno entre vectores
similitud = cosine_similarity(vectores)

c:\Users\Juampi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [17]:
# Se obtiene el array de similitud
similitud[0]

array([1.        , 0.36514837, 0.50709255, ..., 0.51639778, 0.63245553,
       0.54772256])

In [18]:
# Ordenamos la similitud entre más similar a menos similar tomando 5 valores
sorted(list(enumerate(similitud[0])), reverse=True, key=lambda x:x[1])[1:6]

[(1351, 0.9999999999999999),
 (3017, 0.9999999999999999),
 (3356, 0.9999999999999999),
 (3574, 0.9999999999999999),
 (4386, 0.9999999999999999)]

A continuación, se genera una función (recomendacion) que toma como entrada el ID de un juego y devuelve una lista de los cinco juegos más recomendados basándose en la similitud del coseno entre vectores.

La función devuelve la lista de títulos recomendados.

In [22]:
def recomendacion(juego):
    try:
        # Se busca el índice del juego en el DataFrame original (df).
        # Este índice es utilizado para acceder a la fila correspondiente en la matriz de similitud.
        indice_juego = modelo_df[modelo_df["item_id"] == juego].index[0]
        
        # Cálculo de Similitudes: Se obtienen las distancias de similitud entre el juego de entrada y todos los demás juegos en el conjunto de datos. 
        distances = similitud[indice_juego]
        
        # Las distancias se ordenan de manera descendente, y se seleccionan los cinco juegos más similares (excluyendo el juego de entrada)
        lista_juegos = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
        
        # Salida. Devuelve la lista de títulos recomendados.
        recommended_titles = [modelo_df.iloc[i[0]]['name'] for i in lista_juegos]
        
        return recommended_titles
    except IndexError:
        print(f"El juego '{juego}' no se encontró en el DataFrame.")
        return []


In [23]:
# Se aplica la función al dataframe para obtener una nueva columna con las recomendaciones ya que es más facil 
# de cargar y leer para la funcion
modelo_df['RecomendacionesTop5'] = modelo_df['item_id'].apply(recomendacion)

El juego '658870' no se encontró en el DataFrame.
El juego '681550' no se encontró en el DataFrame.


In [ ]:
# Eliminar columnas innecesarias para disminuir el tamaño del archivo de salida
modelo_df.drop(columns=['name', 'genres'], inplace=True)


In [31]:
modelo_df

,item_id,RecomendacionesTop5
0,761140,"[Surgeon Simulator, Urja, Pixel Puzzles 2: Ani..."
1,643980,"[Shadow Hunter, Card Hunter, Immortal Empire, ..."
2,670290,"[Snooker-online multiplayer snooker game!, Mal..."
3,767400,"[Atomic Adam: Episode 1, LEGO® Star Wars™ - Th..."
4,773570,"[Shufflepuck Cantina Deluxe, Canyon Capers - R..."
...,...,...
32128,769330,"[Fate of the World: Migration, Fate of the Wor..."
32129,745400,"[World of Goo, Obulis, Osmos, Fortix, Puzzler ..."
32130,773640,"[Try Hard Parking, Car Mechanic Simulator 2015..."
32131,733530,"[Zen of Sudoku, Gumboy - Crazy Adventures™, Ni..."


In [30]:
#Eliminamos los registros de los juegos que no se encuentran en el df, lso dos ultimos
modelo_df = modelo_df.drop(modelo_df[(modelo_df['item_id'] == 658870) | (modelo_df['item_id'] == 681550)].index)

In [32]:
modelo_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32131 entries, 0 to 32132
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   item_id              32131 non-null  int32 
 1   RecomendacionesTop5  32131 non-null  object
dtypes: int32(1), object(1)
memory usage: 627.6+ KB


In [34]:
# Guardar el DataFrame resultante en un nuevo archivo PARQUET 
modelo_df.to_parquet('C:\\Users\\Juampi\\Desktop\\Henry\\PI_MLOps\\Datasets\\archivos_ML\\recomienda_item_item.parquet', index=False)